In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("/content/drive/MyDrive/ratings.csv")

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
nu=max(df['userId'].unique())

In [5]:
nm=max(df['movieId'].unique())

In [6]:
print(nu,nm)

138493 131262


In [7]:
print(len(df['userId'].unique()),len(df['movieId'].unique()))

138493 26744


In [8]:
x=df['userId']
z=df['rating']

In [9]:
unique_movie_ids = sorted(df['movieId'].unique())
movie_map = {mid: idx for idx, mid in enumerate(unique_movie_ids)}
reverse_movie_map = {idx: mid for mid, idx in movie_map.items()}
df['movie_index'] = df['movieId'].map(movie_map)
df.head()
y=df['movie_index']
nm=max(df['movie_index'].unique())
from scipy.sparse import lil_matrix
matrix = lil_matrix((nu,nm))
for i in range(nu+1):
  matrix[x[i],y[i]] =z[i]

In [10]:
print(matrix[1,541])

0.0


In [11]:
print(matrix.shape)

(138493, 26743)


In [12]:
!pip install lightfm

In [13]:
from lightfm import LightFM
model = LightFM(loss='warp')
model.fit(matrix, epochs=60)

In [14]:
from lightfm.evaluation import precision_at_k, auc_score
# Compute precision at K
train_precision = precision_at_k(model, matrix, k=5).mean()
print(f"Precision@5: {train_precision:.4f}")
# Compute AUC score
train_auc = auc_score(model, matrix).mean()
print(f"AUC Score: {train_auc:.4f}")

Precision@5: 0.5567
AUC Score: 0.9886


In [15]:
print(matrix.shape)

(138493, 26743)


In [16]:
dm=pd.read_csv("/content/drive/MyDrive/movies.csv")

In [17]:
def get_title(movie_id):
  movie_row = dm[dm['movieId'].isin(movie_id)]
  print(movie_row)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def get_similar_movies(movie_id_internal,movie_embeddings = model.item_embeddings, top_k=5):
    movie_id_internal = movie_map [movie_id_internal]
    target_embedding = movie_embeddings[movie_id_internal].reshape(1, -1)
    # Compute cosine similarities only between this movie and all others
    similarities = cosine_similarity(target_embedding, movie_embeddings).flatten()
    # Exclude the movie itself, then get top_k
    similar_ids = np.argsort(-similarities)
    similar_ids = [i for i in similar_ids if i != movie_id_internal][:top_k]
    real_ids =[int(reverse_movie_map[i]) for i in similar_ids]
    get_title(real_ids)

In [27]:
def recomendation(user_id,top_k=5):
  n_items = matrix.shape[1]
  # Predict scores for all movies
  scores = model.predict(user_id, np.arange(n_items))
  # Rank items
  top_items = np.argsort(-scores)[:5]  # Top 5 recommended movie indices
  real_ids =[int(reverse_movie_map[i]) for i in top_items]
  print("Top 5 recommended movieIds:", real_ids)
  get_title(real_ids)

In [20]:
def intrest(user_id,movie_id):
  # User ID for whom we want to predict the score for a particular movie id
  mid_internal = [movie_map[movie_id]]  # if mid is a real movieId
  score = model.predict(user_id, mid_internal)
  return score

In [21]:
def newuser():
  top_movies = df.groupby('movieId')['rating'].mean().sort_values(ascending=False).head(10)
  popular_titles = df[df['movieId'].isin(top_movies.index)]
  get_title(popular_titles['movieId'])

In [22]:
dm.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [23]:
newuser()

       movieId                                           title  \
17590    88488             Summer Wishes, Winter Dreams (1973)   
17739    89133                            Boys (Drenge) (1977)   
19205    95517               Barchester Chronicles, The (1982)   
21744   105187                       Linotype: The Film (2012)   
21746   105191                             Rocaterrania (2009)   
21851   105526  Year Zero: The Silent Death of Cambodia (1979)   
21852   105529                        Stealing a Nation (2004)   
21932   105841                                   B-Side (2013)   
21934   105846                            Only Daughter (2013)   
26403   126945                              Small Roads (2011)   

                    genres  
17590                Drama  
17739                Drama  
19205                Drama  
21744          Documentary  
21746  Documentary|Fantasy  
21851          Documentary  
21852          Documentary  
21932       Comedy|Romance  
21934       

In [24]:
recomendation(6)

Top 5 recommended movieIds: [380, 356, 457, 150, 296]
     movieId                 title                                    genres
148      150      Apollo 13 (1995)                      Adventure|Drama|IMAX
293      296   Pulp Fiction (1994)               Comedy|Crime|Drama|Thriller
352      356   Forrest Gump (1994)                  Comedy|Drama|Romance|War
376      380      True Lies (1994)  Action|Adventure|Comedy|Romance|Thriller
453      457  Fugitive, The (1993)                                  Thriller


In [25]:
get_similar_movies(1)

     movieId                                      title  \
31        32  Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   
315      318           Shawshank Redemption, The (1994)   
352      356                        Forrest Gump (1994)   
476      480                       Jurassic Park (1993)   
591      597                        Pretty Woman (1990)   

                               genres  
31            Mystery|Sci-Fi|Thriller  
315                       Crime|Drama  
352          Comedy|Drama|Romance|War  
476  Action|Adventure|Sci-Fi|Thriller  
591                    Comedy|Romance  


In [31]:
print("1:movie recomendation")
print("2:your intrest on the movive")
print("3: new user")
print("4: to get similar movies")
x=int(input("please select one among them:"))
if x==1:
  user_id=int(input("please enter the user id:"))
  recomendation(user_id)
elif x==2:
  user_id=int(input("please enter the user id:"))
  movie_id=int(input("please enter the movie id:"))
  y=intrest(user_id,movie_id)
  print(y)
  if y<=0:
    print("you dont like it")
  elif y>0 and y<1:
    print("you may not like it")
  elif y>1 and y<2:
    print("you may like it")
  elif y>=2:
    print("you will like it")
elif x==3:
  newuser()
elif x==4:
  movie_id=int(input("please enter the movie id:"))
  get_similar_movies(movie_id)
else:
  print("invalid input")

1:movie recomendation
2:your intrest on the movive
3: new user
4: to get similar movies
please select one among them:4
please enter the movie id:4
      movieId                        title                genres
347       351      Corrina, Corrina (1994)  Comedy|Drama|Romance
1090     1113        Associate, The (1996)                Comedy
1332     1363  Preacher's Wife, The (1996)                 Drama
1377     1410     Evening Star, The (1996)          Comedy|Drama
1578     1632   Smile Like Yours, A (1997)        Comedy|Romance
